In [21]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/skin-cancer-malignant-vs-benign


In [40]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os

In [23]:
# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images
    transforms.RandomHorizontalFlip(),  # Horizontal flip
    transforms.RandomRotation(20),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])
val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [24]:
# Kaggle dataset paths
data_dir = "/kaggle/input/skin-cancer-malignant-vs-benign"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

In [25]:
# Load training dataset
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)

In [26]:
# Split the dataset into training and validation sets
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])


In [32]:
class TransformDataset:
    def __init__(self, subset, transform, dataset):
        self.subset = subset
        self.transform = transform
        self.dataset = dataset  # Original dataset for accessing image paths

    def __getitem__(self, index):
        # Get the raw image path and label from the original dataset
        idx_in_dataset = self.subset.indices[index]  # Map subset index to original dataset index
        image_path, label = self.dataset.imgs[idx_in_dataset]
        
        # Open the image using PIL and apply the transformation
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image at index {index}: {e}")
            raise  # Re-raise the exception for debugging
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

    def __len__(self):
        return len(self.subset)


In [34]:
# Apply validation transformations to the validation subset
val_dataset = TransformDataset(val_subset, val_transform, train_dataset)

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
# For test data
test_dataset = datasets.ImageFolder(test_dir, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Print dataset information
print(f"Training samples: {len(train_subset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Class names: {train_dataset.classes}")


Training samples: 2109
Validation samples: 528
Test samples: 660
Class names: ['benign', 'malignant']


In [35]:
# Define the SimpleCNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128),  # Adjust based on input size
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Initialize the model
model = SimpleCNN()


In [36]:
# Move the model to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

SimpleCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=1, bias=True)
    (4): Sigmoid()
  )
)

In [37]:


# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                preds = (outputs >= 0.5).float()
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {correct/total:.4f}')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>self._shutdown_workers()
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
        self._shutdown_workers()if w.is_alive():

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
        if w.is_alive():assert self._parent_pid == os.getpid(), 'can only test a child process'

  File "/usr/lib/python3.10/multiprocessing/process.py", line 1

Epoch [1/10], Loss: 0.6427


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>if w.is_alive():

Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    assert self._parent_pid == os.getpid(), 'can only test a child process'    
self._shutdown_workers()AssertionError
: can only test a child process  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers

    if w.is_alive():Exception ignored 

Validation Loss: 0.5106, Accuracy: 0.7424


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    Exception ignored in: if w.is_alive():<function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>

Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
        assert self._parent_pid == os.getpid(), 'can only test a child process'
self._shutdown_workers()
AssertionError  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
: can only test a child process    
if w.is_alive():Exception ignored 

...

Validation Loss: 0.3529, Accuracy: 0.8485


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>self._shutdown_workers()

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    if w.is_alive():    
self._shutdown_workers()  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive

      File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
assert self._parent_pid == os.getpid(), 'can only test a child process'    
if w.is_alive():AssertionError: 
can only test a child process  File "/usr/lib/p

Epoch [10/10], Loss: 0.4091


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    Exception ignored in: if w.is_alive():<function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>

  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
Traceback (most recent call last):
      File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
assert self._parent_pid == os.getpid(), 'can only test a child process'    
self._shutdown_workers()AssertionError
:   File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
can only test a child process
    Exception ignored in: if w.is_aliv

Validation Loss: 0.3810, Accuracy: 0.8485


In [39]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f"Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {accuracy:.4f}")

# Evaluate the model on the test set
evaluate_model(model, test_loader, criterion)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7b6a3d93edd0>  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__

    Traceback (most recent call last):
self._shutdown_workers()  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__

      File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
self._shutdown_workers()    
if w.is_alive():  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers

      File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():    assert self._parent_pid == os.getpid(), 'can only test a child process'

AssertionError  File "/usr/lib/python3.10/multiprocessing/proce

Test Loss: 0.3977, Test Accuracy: 0.8000


In [41]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'skin_cancer_model.pth')
print("Model saved successfully.")

Model saved successfully.


In [42]:
# Load the model's state dictionary
model = SimpleCNN()  # Or use the same model architecture
model.load_state_dict(torch.load('skin_cancer_model.pth'))
model.to(device)
model.eval()
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
from PIL import Image

def predict_image(image_path, model, transform):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    
    # Get the prediction
    model.eval()
    with torch.no_grad():
        output = model(image)
        prob = output.item()  # Probability of being malignant
        label = "Malignant" if prob >= 0.5 else "Benign"
    
    return prob, label

# Example usage
image_path = "/path/to/test/image.jpg"
prob, label = predict_image(image_path, model, val_transform)
print(f"Predicted Label: {label}, Probability: {prob:.4f}")

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, test_loader, transform, num_images=5):
    model.eval()
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            preds = (outputs >= 0.5).float()
            
            for i in range(num_images):
                image = inputs[i].cpu().numpy().transpose((1, 2, 0))  # Convert to HWC
                image = image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # Un-normalize
                image = image.clip(0, 1)
                
                true_label = "Malignant" if labels[i].item() >= 0.5 else "Benign"
                pred_label = "Malignant" if preds[i].item() >= 0.5 else "Benign"
                
                axes[i].imshow(image)
                axes[i].set_title(f"True: {true_label}\nPred: {pred_label}")
                axes[i].axis('off')
            plt.show()
            break

# Visualize predictions
visualize_predictions(model, test_loader, val_transform, num_images=5)

In [ ]:
import gradio as gr

def classify_image(image):
    image = val_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        prob = output.item()
        label = "Malignant" if prob >= 0.5 else "Benign"
    return {label: prob}

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="Skin Cancer Classifier",
    description="Upload an image to classify it as benign or malignant."
)

interface.launch()